In [31]:
import pandas as pd
import numpy as np
import datetime as dt

import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *#avg, count, expr
from pyspark.sql.types import *
from pyspark.ml.feature import VectorAssembler, StringIndexer, MinMaxScaler
from pyspark.ml.classification import *
from pyspark.ml.evaluation import *
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import mlflow
from mlflow import pyspark
from mlflow.tracking import MlflowClient

In [2]:
# initialize
sc = pyspark.SparkContext()
spark = SparkSession(sc)
spark.sparkContext.appName = 'exploreReadWrite'
# show the number of cores
print('%d cores'%spark._jsc.sc().getExecutorMemoryStatus().keySet().size())
spark

1 cores


In [3]:
# load the data
fil = '../data/Toddler_Autism_dataset_July_2018.csv'
schem = StructType([StructField('Case_No', IntegerType()), StructField('A1', IntegerType()), StructField('A2', IntegerType()),
                    StructField('A3', IntegerType()), StructField('A4', IntegerType()), StructField('A5', IntegerType()),
                    StructField('A6', IntegerType()), StructField('A7', IntegerType()), StructField('A8', IntegerType()),
                    StructField('A9', IntegerType()), StructField('A10', IntegerType()), StructField('Age_Mons', IntegerType()),
                    StructField('Qchat-10-Score', IntegerType()), StructField('Sex', StringType()), StructField('Ethnicity', StringType()),
                    StructField('Jaundice', StringType()), StructField('Family_mem_with_ASD', StringType()),
                    StructField('Who completed the test', StringType()), StructField('Class/ASD Traits ', StringType())])

asd = spark.read.format('csv').options(header=True).schema(schem).load(fil)
print('%d records'%asd.count())
asd.show(truncate=False)

1054 records
+-------+---+---+---+---+---+---+---+---+---+---+--------+--------------+---+--------------+--------+-------------------+------------------------+-----------------+
|Case_No|A1 |A2 |A3 |A4 |A5 |A6 |A7 |A8 |A9 |A10|Age_Mons|Qchat-10-Score|Sex|Ethnicity     |Jaundice|Family_mem_with_ASD|Who completed the test  |Class/ASD Traits |
+-------+---+---+---+---+---+---+---+---+---+---+--------+--------------+---+--------------+--------+-------------------+------------------------+-----------------+
|1      |0  |0  |0  |0  |0  |0  |1  |1  |0  |1  |28      |3             |f  |middle eastern|yes     |no                 |family member           |No               |
|2      |1  |1  |0  |0  |0  |1  |1  |0  |0  |0  |36      |4             |m  |White European|yes     |no                 |family member           |Yes              |
|3      |1  |0  |0  |0  |0  |0  |1  |1  |0  |1  |36      |4             |m  |middle eastern|yes     |no                 |family member           |Yes             

In [4]:
responseVar = 'Class/ASD Traits '
asd.groupBy(responseVar).count().show()

+-----------------+-----+
|Class/ASD Traits |count|
+-----------------+-----+
|               No|  326|
|              Yes|  728|
+-----------------+-----+



In [5]:
''' prep the data for modeling '''
# set inputs
inputColumns = asd.columns[1:-1]
strFeats = {c:c+'_int' for c in inputColumns if asd.schema[c].dataType is StringType()}
inputColumns = [strFeats.get(c, c) for c in inputColumns]

# create all numerical features & numerical response
indxr = StringIndexer(inputCols=list(strFeats.keys()) + ['label_str'], outputCols=list(strFeats.values()) + ['label'])
asdML = asd.withColumnRenamed(responseVar, 'label_str')
asdML = indxr.fit(asdML).transform(asdML)

# create the features vector
assr = VectorAssembler(inputCols=inputColumns, outputCol='features')
asdML = assr.transform(asdML)

# talk
display(asdML.limit(10).toPandas())
asdML.select('features', 'label').show(truncate=True)
asdML.select('features').take(1)
print('First row features = %s'%asdML.select('features').take(1)[0])

,Case_No,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,Family_mem_with_ASD,Who completed the test,label_str,label,Jaundice_int,Family_mem_with_ASD_int,Who completed the test_int,Ethnicity_int,Sex_int,features
0,1,0,0,0,0,0,0,1,1,0,...,no,family member,No,1.0,1.0,0.0,0.0,2.0,1.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, ..."
1,2,1,1,0,0,0,1,1,0,0,...,no,family member,Yes,0.0,1.0,0.0,0.0,0.0,0.0,"(1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ..."
2,3,1,0,0,0,0,0,1,1,0,...,no,family member,Yes,0.0,1.0,0.0,0.0,2.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, ..."
3,4,1,1,1,1,1,1,1,1,1,...,no,family member,Yes,0.0,0.0,0.0,0.0,5.0,0.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
4,5,1,1,0,1,1,1,1,1,1,...,yes,family member,Yes,0.0,0.0,1.0,0.0,0.0,1.0,"[1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
5,6,1,1,0,0,1,1,1,1,1,...,no,family member,Yes,0.0,0.0,0.0,0.0,4.0,0.0,"[1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
6,7,1,0,0,1,1,1,0,0,1,...,no,family member,Yes,0.0,1.0,0.0,0.0,1.0,0.0,"(1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, ..."
7,8,0,1,0,0,1,0,1,1,1,...,no,family member,Yes,0.0,1.0,0.0,0.0,1.0,0.0,"(0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, ..."
8,9,0,0,0,0,0,0,1,0,0,...,no,family member,No,1.0,0.0,0.0,0.0,1.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
9,10,1,1,1,0,1,1,0,1,1,...,no,Health Care Professional,Yes,0.0,0.0,0.0,1.0,3.0,0.0,"[1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, ..."


+--------------------+-----+
|            features|label|
+--------------------+-----+
|(17,[6,7,9,10,11,...|  1.0|
|(17,[0,1,5,6,10,1...|  0.0|
|(17,[0,6,7,9,10,1...|  0.0|
|[1.0,1.0,1.0,1.0,...|  0.0|
|[1.0,1.0,0.0,1.0,...|  0.0|
|[1.0,1.0,0.0,0.0,...|  0.0|
|(17,[0,3,4,5,8,10...|  0.0|
|(17,[1,4,6,7,8,9,...|  0.0|
|(17,[6,9,10,11,13...|  1.0|
|[1.0,1.0,1.0,0.0,...|  0.0|
|[1.0,0.0,0.0,1.0,...|  0.0|
|[1.0,1.0,1.0,1.0,...|  0.0|
|(17,[10,12,13,14]...|  1.0|
|[1.0,1.0,1.0,1.0,...|  0.0|
|(17,[10,13],[18.0...|  1.0|
|(17,[0,1,2,4,6,7,...|  0.0|
|(17,[10,13,15],[3...|  1.0|
|[1.0,1.0,1.0,0.0,...|  0.0|
|(17,[0,4,9,10,11,...|  1.0|
|[1.0,1.0,1.0,0.0,...|  0.0|
+--------------------+-----+
only showing top 20 rows

First row features = (17,[6,7,9,10,11,12,13,14],[1.0,1.0,1.0,28.0,3.0,1.0,2.0,1.0])


In [6]:
''' split for cross-val '''
trainPerc = 0.7
randSeed = 42
trainASD, testASD = asdML.randomSplit([trainPerc, 1.0 - trainPerc], seed=randSeed)

# talk
print('Training Cases')
trainASD.select('Case_No').show()
print('Testing Cases')
testASD.select('Case_No').show()

Training Cases
+-------+
|Case_No|
+-------+
|      1|
|      2|
|      4|
|      5|
|      6|
|      8|
|     11|
|     12|
|     13|
|     17|
|     18|
|     19|
|     21|
|     23|
|     26|
|     27|
|     28|
|     32|
|     34|
|     37|
+-------+
only showing top 20 rows

Testing Cases
+-------+
|Case_No|
+-------+
|      3|
|      7|
|      9|
|     10|
|     14|
|     15|
|     16|
|     20|
|     22|
|     24|
|     25|
|     29|
|     30|
|     31|
|     33|
|     35|
|     36|
|     40|
|     43|
|     44|
+-------+
only showing top 20 rows



### Logistic Regression

In [7]:
''' try logistic regression - using predefined train/test split '''
# create objects
#auc = BinaryClassificationEvaluator(metricName='areaUnderROC')
acc = MulticlassClassificationEvaluator(metricName='accuracy')
logreg = LogisticRegression()

# train & eval
fitModel = logreg.fit(trainASD.select('features', 'label'))
trainRes = fitModel.evaluate(trainASD.select('features', 'label'))
trainAcc = acc.evaluate(trainRes.predictions)

# now evaluate test accuracy
testRes = fitModel.transform(testASD.select('features', 'label'))
testAcc = acc.evaluate(testRes)

print('Train Accuracy = %0.3f, Test Accurcy = %0.3f'%(trainAcc, testAcc))

Train Accuracy = 1.000, Test Accurcy = 1.000


In [8]:
''' now use the builtin cross-validator '''
#estimator
logreg = LogisticRegression()
# parameters grid
params = (ParamGridBuilder().addGrid(logreg.threshold, [0.4, 0.5, 0.6]).addGrid(logreg.elasticNetParam, [0.0, 0.25, 0.5, 0.75, 1.0]).build())
# cross-validator
cv = CrossValidator(estimator=logreg, estimatorParamMaps=params, evaluator=acc, numFolds=5)

# run
fitModel = cv.fit(trainASD.select('features', 'label'))

In [10]:
bestModel = fitModel.bestModel

# best model parameters
print('Threshold = %0.3f, Elastic Net = %0.3f'%(bestModel.getThreshold(), bestModel.getElasticNetParam()))

# show performance of best model
testAcc = acc.evaluate(bestModel.transform(testASD.select('features', 'label')))
print('Best Model Test Accuracy = %0.3f'%testAcc)

Threshold = 0.400, Elastic Net = 0.000
Best Model Test Accuracy = 1.000


In [11]:
# only available for logistic regression
summ = bestModel.summary
summ.predictions.describe().show()
summ.objectiveHistory

print('Summary Information %s'%[am for am in dir(summ) if am[0] != '_'])

+-------+-------------------+-------------------+
|summary|              label|         prediction|
+-------+-------------------+-------------------+
|  count|                780|                780|
|   mean|0.30256410256410254|0.30256410256410254|
| stddev| 0.4596628666274773| 0.4596628666274773|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+

Summary Information ['accuracy', 'areaUnderROC', 'fMeasureByLabel', 'fMeasureByThreshold', 'falsePositiveRateByLabel', 'featuresCol', 'labelCol', 'labels', 'objectiveHistory', 'pr', 'precisionByLabel', 'precisionByThreshold', 'predictionCol', 'predictions', 'probabilityCol', 'recallByLabel', 'recallByThreshold', 'roc', 'scoreCol', 'totalIterations', 'truePositiveRateByLabel', 'weightCol', 'weightedFMeasure', 'weightedFalsePositiveRate', 'weightedPrecision', 'weightedRecall', 'weightedTruePositiveRate']


### Decision Tree

In [17]:
''' now use the builtin cross-validator '''
#estimator
estim = DecisionTreeClassifier()
# parameters grid
params = (ParamGridBuilder().addGrid(estim.maxBins, [20, 40, 80, 100])\
              .addGrid(estim.maxDepth, [5, 10, 30]).build())
# cross-validator
cv = CrossValidator(estimator=estim, estimatorParamMaps=params, evaluator=acc, numFolds=5)

# run
fitModel = cv.fit(trainASD.select('features', 'label'))

In [19]:
bestModel = fitModel.bestModel

# best model parameters
print('Max Quant Bins = %0.3f, Max Depth = %0.3f'%(bestModel.getMaxBins(), bestModel.getMaxDepth()))

# show performance of best model
testAcc = acc.evaluate(bestModel.transform(testASD.select('features', 'label')))
print('Best Model Test Accuracy = %0.3f'%testAcc)

# view feature importances
imports = bestModel.featureImportances.toArray()
for (col, imp) in zip(inputColumns, imports):
    print('%s feature importance = %0.3f'%(col, imp))

Max Quant Bins = 20.000, Max Depth = 5.000
Best Model Test Accuracy = 1.000
A1 feature importance = 0.000
A2 feature importance = 0.000
A3 feature importance = 0.000
A4 feature importance = 0.000
A5 feature importance = 0.000
A6 feature importance = 0.000
A7 feature importance = 0.000
A8 feature importance = 0.000
A9 feature importance = 0.000
A10 feature importance = 0.000
Age_Mons feature importance = 0.000
Qchat-10-Score feature importance = 1.000
Sex_int feature importance = 0.000
Ethnicity_int feature importance = 0.000
Jaundice_int feature importance = 0.000
Family_mem_with_ASD_int feature importance = 0.000
Who completed the test_int feature importance = 0.000


In [21]:
# what is available in the model object
print([i for i in dir(bestModel) if i[0] != '_'])

['cacheNodeIds',
 'checkpointInterval',
 'clear',
 'copy',
 'depth',
 'explainParam',
 'explainParams',
 'extractParamMap',
 'featureImportances',
 'featuresCol',
 'getCacheNodeIds',
 'getCheckpointInterval',
 'getFeaturesCol',
 'getImpurity',
 'getLabelCol',
 'getLeafCol',
 'getMaxBins',
 'getMaxDepth',
 'getMaxMemoryInMB',
 'getMinInfoGain',
 'getMinInstancesPerNode',
 'getMinWeightFractionPerNode',
 'getOrDefault',
 'getParam',
 'getPredictionCol',
 'getProbabilityCol',
 'getRawPredictionCol',
 'getSeed',
 'getThresholds',
 'getWeightCol',
 'hasDefault',
 'hasParam',
 'impurity',
 'isDefined',
 'isSet',
 'labelCol',
 'leafCol',
 'load',
 'maxBins',
 'maxDepth',
 'maxMemoryInMB',
 'minInfoGain',
 'minInstancesPerNode',
 'minWeightFractionPerNode',
 'numClasses',
 'numFeatures',
 'numNodes',
 'params',
 'predict',
 'predictLeaf',
 'predictProbability',
 'predictRaw',
 'predictionCol',
 'probabilityCol',
 'rawPredictionCol',
 'read',
 'save',
 'seed',
 'set',
 'setFeaturesCol',
 'setLe

### Random Forest

In [22]:
''' now use the builtin cross-validator '''
#estimator
estim = RandomForestClassifier(numTrees=20)
# parameters grid
params = (ParamGridBuilder().addGrid(estim.maxBins, [20, 40, 80, 100])\
              .addGrid(estim.maxDepth, [5, 10, 30]).build())
# cross-validator
cv = CrossValidator(estimator=estim, estimatorParamMaps=params, evaluator=acc, numFolds=5)

# run
fitModel = cv.fit(trainASD.select('features', 'label'))

In [23]:
bestModel = fitModel.bestModel

# best model parameters
print('Max Quant Bins = %0.3f, Max Depth = %0.3f'%(bestModel.getMaxBins(), bestModel.getMaxDepth()))

# show performance of best model
testAcc = acc.evaluate(bestModel.transform(testASD.select('features', 'label')))
print('Best Model Test Accuracy = %0.3f'%testAcc)

# view feature importances
imports = bestModel.featureImportances.toArray()
for (col, imp) in zip(inputColumns, imports):
    print('%s feature importance = %0.3f'%(col, imp))

Max Quant Bins = 20.000, Max Depth = 5.000
Best Model Test Accuracy = 1.000
A1 feature importance = 0.030
A2 feature importance = 0.029
A3 feature importance = 0.007
A4 feature importance = 0.041
A5 feature importance = 0.040
A6 feature importance = 0.041
A7 feature importance = 0.080
A8 feature importance = 0.006
A9 feature importance = 0.071
A10 feature importance = 0.003
Age_Mons feature importance = 0.002
Qchat-10-Score feature importance = 0.646
Sex_int feature importance = 0.000
Ethnicity_int feature importance = 0.003
Jaundice_int feature importance = 0.000
Family_mem_with_ASD_int feature importance = 0.000
Who completed the test_int feature importance = 0.001


In [24]:
# what is available in the model object
print([i for i in dir(bestModel) if i[0] != '_'])

['bootstrap', 'cacheNodeIds', 'checkpointInterval', 'clear', 'copy', 'evaluate', 'explainParam', 'explainParams', 'extractParamMap', 'featureImportances', 'featureSubsetStrategy', 'featuresCol', 'getBootstrap', 'getCacheNodeIds', 'getCheckpointInterval', 'getFeatureSubsetStrategy', 'getFeaturesCol', 'getImpurity', 'getLabelCol', 'getLeafCol', 'getMaxBins', 'getMaxDepth', 'getMaxMemoryInMB', 'getMinInfoGain', 'getMinInstancesPerNode', 'getMinWeightFractionPerNode', 'getNumTrees', 'getOrDefault', 'getParam', 'getPredictionCol', 'getProbabilityCol', 'getRawPredictionCol', 'getSeed', 'getSubsamplingRate', 'getThresholds', 'getWeightCol', 'hasDefault', 'hasParam', 'hasSummary', 'impurity', 'isDefined', 'isSet', 'labelCol', 'leafCol', 'load', 'maxBins', 'maxDepth', 'maxMemoryInMB', 'minInfoGain', 'minInstancesPerNode', 'minWeightFractionPerNode', 'numClasses', 'numFeatures', 'numTrees', 'params', 'predict', 'predictLeaf', 'predictProbability', 'predictRaw', 'predictionCol', 'probabilityCol',

### Now try mlflow

In [38]:
experiment_name = 'My Experiment'
mlflow.set_experiment(experiment_name='My Experiment')
mlfc = MlflowClient()

experiments = mlfc.list_experiments()

def create_run(expName, mlFlowClient):
    mlflow.set_experiment(experiment_name='My Experiment')
    experiments = mlFlowClient.list_experiments()
    for exp in experiments:
        if expName in exp.name:
            expIndex = experiments.index(exp)
            run = mlFlowClient.create_run(experiments[expIndex].experiment_id)
            print(run)
            return run

In [46]:
''' use mlflow with the cross validator now '''
run = create_run(experiment_name, mlfc)

#estimator
estim = LogisticRegression()
estimName = 'LogisticReg'
# parameters grid
params = (ParamGridBuilder().addGrid(estim.threshold, [0.4, 0.5, 0.6]).addGrid(estim.elasticNetParam, [0.0, 0.25, 0.5, 0.75, 1.0]).build())
paramNames = ['threshold', 'elasticnetparam']
# cross-validator
cv = CrossValidator(estimator=logreg, estimatorParamMaps=params, evaluator=acc, numFolds=5)
# run
fitModel = cv.fit(trainASD.select('features', 'label'))

# get results
bestModel = fitModel.bestModel
# best model parameters
print('Threshold = %0.3f, Elastic Net = %0.3f'%(bestModel.getThreshold(), bestModel.getElasticNetParam()))
# show performance of best model
testAcc = acc.evaluate(bestModel.transform(testASD.select('features', 'label')))
print('Best Model Test Accuracy = %0.3f'%testAcc)
# get best parameters
bestParams = bestModel.extractParamMap()

# add stuff to mlflow
mlfc.set_tag(run.info.run_id, 'Estimator', estimName)
mlfc.set_tag(run.info.run_id, 'PRNG Seed', randSeed)
mlfc.set_tag(run.info.run_id, 'Train Perc', trainPerc)
for (key, val) in bestParams.items():
    for parm in paramNames:
        if parm in key.name.lower():
            mlfc.log_param(run.info.run_id, parm, val)
mlfc.log_metric(run.info.run_id, 'Accuracy', testAcc)
mlfc.set_terminated(run.info.run_id)

<Run: data=<RunData: metrics={}, params={}, tags={}>, info=<RunInfo: artifact_uri='file:///home/ahowe42/pyspark_explore/pyspark_essentials_udemy/mlruns/1/fd5f0a7fdafe4928994faead5b122bef/artifacts', end_time=None, experiment_id='1', lifecycle_stage='active', run_id='fd5f0a7fdafe4928994faead5b122bef', run_uuid='fd5f0a7fdafe4928994faead5b122bef', start_time=1622235063594, status='RUNNING', user_id='unknown'>>
Threshold = 0.500, Elastic Net = 0.000
Best Model Test Accuracy = 1.000


In [47]:
sc.stop()